In [20]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。
# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
import torch
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


Load model

In [19]:
import torch
config = EcgConfig()    
model = torch.load("..\\model_save\\model_epoch_98.pth")
model.eval()

ECGformer(
  (encoder): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (queries_projection): Linear(in_features=192, out_features=192, bias=True)
            (values_projection): Linear(in_features=192, out_features=192, bias=True)
            (keys_projection): Linear(in_features=192, out_features=192, bias=True)
            (final_projection): Linear(in_features=192, out_features=192, bias=True)
          )
          (2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MLP(
            (0): Linear(in_features=192, out_features=768, bias=True)
            (1): GELU(approximate='none')
            (2): Linear(in_features=768, out_features=192, bias=True)
          )
          (2):

量化模型

In [21]:
import torch.quantization

# 使用 Eager Mode Quantization
# 將 torch.nn.Linear 的參數映射到 -127~127 之間
model.to('cpu')
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

torch.save(quantized_model, "..\\model_save\\model_quantized_98.pth")

準確度測試

In [22]:
print(quantized_model)

ECGformer(
  (encoder): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (queries_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (values_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (keys_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (final_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)


In [24]:
import einops
loader = get_data_loaders(DatasetConfig())
accuracy = 0
quantized_model.to('cpu')
for signal, label in loader[Mode.train]:
    signal.to('cpu')
    label.to('cpu')
    signal = einops.rearrange(signal, "b c e -> b e c")
    # print(signal)
    p = quantized_model(signal)
    print(p)
    print(label)
    print(signal.shape, label.shape)
    print(p.argmax(1) == label)
    accuracy += torch.sum(p.argmax(1) == label)
    print(f"accuracy: {accuracy / config.dataset.batch_size}")
    break



tensor([[  5.3291,   1.3786,  -2.6988,   1.3100, -10.6793,  -2.9013],
        [ -1.5244,  -0.2526,   9.1004,   1.1558,  -0.3941,  -8.7845],
        [  0.8906,  -1.7551,   3.3844,  -0.8985,  -3.1181,  -2.6377],
        [  8.4686,   0.3487,  -8.8782,  -2.0669,  -4.7619,  -0.9348],
        [  1.5206,  -0.0603,  -0.7347,  -5.8583,  10.2529,  -3.0687],
        [  0.9960,  -2.4833,  -2.2504,  -2.1198,  -2.4565,   6.2091],
        [  5.9401,   1.7555,  -8.0376,  -1.2204, -14.7936,   6.6824],
        [  8.9407,   0.7511,  -2.4705,  -3.8327, -12.3787,  -2.7889],
        [  6.9146,  -0.4525,  -3.8277,  -2.6514,  -4.6309,  -3.0050],
        [  8.3848,   3.9803,  -3.6459,  -6.7881,  -8.0620,  -2.9932],
        [  4.6729,  -1.9720,  -0.4228,  -1.8202,  -4.9414,  -1.9157],
        [  8.3384,   0.8334,  -5.9958,  -2.3623, -13.5658,   1.2096],
        [  1.9265,  -2.6329,  -5.1888,   0.0155,  -7.3862,   7.9568],
        [  5.6045,  -0.1662,  -3.8674,  -1.8391,  -5.7915,  -1.5313],
        [  4.0892,  

## Parameter 提取

In [23]:
for parm in model.parameters():
    print(parm)

Parameter containing:
tensor([[ 0.7767,  0.4179, -0.9639,  ...,  1.0551,  0.6087,  0.1514],
        [-0.7247,  0.7949,  0.6012,  ..., -0.0981, -0.7312, -1.5987],
        [ 2.1631, -0.8226, -1.1655,  ..., -0.3369, -0.1976,  0.3002],
        ...,
        [ 0.2282,  0.4482, -0.0295,  ...,  0.1569, -0.5706, -0.3451],
        [ 0.2163, -0.3091, -1.6473,  ..., -1.4622,  0.0603,  0.7738],
        [-1.3462, -0.3745, -1.0837,  ...,  0.3060, -0.3337,  0.5932]],
       requires_grad=True)
Parameter containing:
tensor([0.9750, 1.0008, 1.0394, 0.9690, 1.0381, 0.9551, 1.0361, 0.9804, 1.0251,
        0.9770, 0.9904, 1.0317, 1.0246, 0.9874, 1.0160, 0.9927, 1.0636, 0.9876,
        0.9794, 1.0233, 1.0472, 1.0000, 1.0550, 1.0249, 0.9639, 1.0036, 1.0061,
        1.0274, 1.0092, 1.0298, 1.0008, 1.0026, 1.0025, 1.0162, 0.9898, 1.0281,
        0.9993, 1.0172, 1.0028, 0.9791, 1.0002, 0.9492, 0.9808, 1.0047, 1.0121,
        1.0756, 1.0484, 1.0530, 1.0308, 1.0548, 1.0228, 0.9818, 1.0365, 1.0565,
        1.0072,